In [3]:
import pandas as pd
import random
import nltk
import re
import numpy as np
import os
import string
import gensim
import ast
import pickle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
#Most Common 50 ADJs (to skip this step, use adjs.pickle)
cleanv2 = pd.read_csv('clean_v2.csv')
    
def feature_common_adjs(words):
    words = eval(words)
    tagged = nltk.pos_tag(words)
    adjs = [x for x, y in tagged if y.startswith('JJ') and x.isalpha() and len(x)>2]
    common_adjs = [(x,y) for x, y in nltk.FreqDist(adjs).most_common(5)]
    return {'most_common_adjs' : common_adjs}

cleanv2['ADJs'] = cleanv2['New'].apply(lambda post: feature_common_adjs(post))

with open('adjs.pickle', 'wb') as handle:
    pickle.dump(cleanv2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
#to skip this step, use adjs_v2.pickle
cleanv2 = pd.read_pickle('adjs.pickle')

wordlist = {}

for i in cleanv2['ADJs']:
    for zipped in i.values():
        for word,count in zipped:
            if word not in wordlist.keys():
                wordlist[word] = count
            else:
                wordlist[word] += count

wordlist = dict(sorted(wordlist.items(), key=lambda item: item[1], reverse = True))
common_adjs = [i for i in wordlist.keys()][:50]

def adj_extractor(post):
    feature_adj = {}
    lst = eval(post)
    lst = list(set(lst))
    for i in common_adjs:
        if i in lst:
            feature_adj[i] = 1
        else:
            feature_adj[i] = 0
    return(feature_adj)

cleanv2['common_adjs'] = cleanv2['New'].apply(lambda x: adj_extractor(x))

with open('adjs_v2.pickle', 'wb') as handle:
    pickle.dump(cleanv2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [61]:
adjDF = pd.read_pickle('adjs_v2.pickle')
featureDF = pd.read_pickle('PickleFeatureSet')

In [62]:
#Pulling values from dict of certain columns
featureDF['ADJs'] = [j for i in featureDF['ADJs'] for j in i.values()]
featureDF['AvgSentLength'] = [j for i in featureDF['AvgSentLength'] for j in i.values()]
featureDF['AvgChar'] = [j for i in featureDF['AvgChar'] for j in i.values()]

# Most common word
most_com = []
for i in featureDF['MostCommonWord']:
    if len(i) == 0:
        most_com.append(0)
    else:
        for j in i.values():
            most_com.append(j)
featureDF['MostCommonWord'] = most_com

# Average TIIDF
featureDF['Avg TFIDF'] = [j for i in featureDF['Avg TFIDF'] for j in i.values()]
avg_tfidf = []
for i in featureDF['Avg TFIDF']:
    if i < 1:
        avg_tfidf.append(i)
    else:
        avg_tfidf.append(0)

featureDF['Avg TFIDF'] = avg_tfidf

# vaderScore: convert the attribute to be a number, negative = 1/neutral = 2/positive = 3
vader_list = []
for i in featureDF['vaderScore']:
    lst = list(i.values())
    attribute = lst.index(max(lst[:2]))
    if attribute == 0:
        vader_list.append(1)  # negative
    elif attribute == 1:
        vader_list.append(2) # neutral
    else:
        vader_list.append(3) # positive
featureDF['vaderScore'] = vader_list
featureDF.head()

,type,EmojiFeature,ADJs,AvgSentLength,MostCommonWord,AvgChar,Avg TFIDF,vaderScore,CombineDict
0,INFJ,"{'< happy >': False, '< laugh >': False, '< fr...",59,13.6,92,5.75,0.042,2,"({'< happy >': False, '< laugh >': False, '< f..."
1,ENTP,"{'< happy >': True, '< laugh >': False, '< fro...",104,11.8,112,5.52,0.023,2,"({'< happy >': True, '< laugh >': False, '< fr..."
2,INTP,"{'< happy >': True, '< laugh >': False, '< fro...",67,14.3,92,5.84,0.032,2,"({'< happy >': True, '< laugh >': False, '< fr..."
3,INTJ,"{'< happy >': False, '< laugh >': False, '< fr...",99,11.9,119,5.93,0.029,2,"({'< happy >': False, '< laugh >': False, '< f..."
4,ENTJ,"{'< happy >': False, '< laugh >': True, '< fro...",67,11.5,97,5.88,0.032,2,"({'< happy >': False, '< laugh >': True, '< fr..."


In [63]:
#This we keep because Austin has already used it in his deep learning model
list_a = list(enumerate(adjDF['common_adjs'][0].keys()))
for index,lab in list_a:
    adjDF[lab] = adjDF['common_adjs'].apply(lambda x: [v for k,v in x.items()][index])
adjDF.head()

,Unnamed: 0,type,Clean,New,ADJs,common_adjs,good,happy,much,cheeky,...,able,strong,human,enfp,interested,entp,emotional,easy,young,certain
0,0,1,' and intj moments sportscenter not top ten...,"['intj', 'moment', 'sportscenter', 'top', 'ten...","{'most_common_adjs': [('good', 3), ('intj', 2)...","{'good': 1, 'happy': 0, 'much': 1, 'cheeky': 1...",1,0,1,1,...,0,0,0,1,0,1,0,0,0,0
1,1,0,'I'm finding the lack of me in these posts ver...,"['m', 'find', 'lack', 'post', 'alarmingsex', '...","{'most_common_adjs': [('big', 11), ('happy', 5...","{'good': 1, 'happy': 1, 'much': 1, 'cheeky': 1...",1,1,1,1,...,1,0,0,0,0,1,0,1,0,0
2,2,1,"'Good one _____ course, to which I say I k...","['good', 'one', '_____', 'course', 'say', 'kno...","{'most_common_adjs': [('good', 7), ('happy', 7...","{'good': 1, 'happy': 1, 'much': 1, 'cheeky': 0...",1,1,1,0,...,1,0,1,0,1,0,0,0,0,0
3,3,1,"'Dear INTP, I enjoyed our conversation the o...","['dear', 'intp', 'enjoyed', 'conversation', 'd...","{'most_common_adjs': [('social', 5), ('much', ...","{'good': 0, 'happy': 1, 'much': 1, 'cheeky': 0...",0,1,1,0,...,1,0,0,0,0,0,1,0,0,1
4,4,0,'You're fired.That's another silly misconcepti...,"['you', 're', 'firedthat', 's', 'another', 'si...","{'most_common_adjs': [('good', 6), ('best', 2)...","{'good': 1, 'happy': 0, 'much': 1, 'cheeky': 0...",1,0,1,0,...,0,0,0,0,1,0,0,1,0,0


In [64]:
#Dropping all other columns except for oneHot and Clean
oneHotADJ = adjDF.drop(['Unnamed: 0', 'type','New', 'ADJs', 'common_adjs'], axis = 1)
oneHotADJ.head()

,Clean,good,happy,much,cheeky,playful,big,sure,many,little,...,able,strong,human,enfp,interested,entp,emotional,easy,young,certain
0,' and intj moments sportscenter not top ten...,1,0,1,1,1,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
1,'I'm finding the lack of me in these posts ver...,1,1,1,1,0,1,0,1,1,...,1,0,0,0,0,1,0,1,0,0
2,"'Good one _____ course, to which I say I k...",1,1,1,0,0,1,0,0,0,...,1,0,1,0,1,0,0,0,0,0
3,"'Dear INTP, I enjoyed our conversation the o...",0,1,1,0,1,0,1,1,0,...,1,0,0,0,0,0,1,0,0,1
4,'You're fired.That's another silly misconcepti...,1,0,1,0,0,1,1,1,1,...,0,0,0,0,1,0,0,1,0,0


In [65]:
#Combinding the Two DF together
df=pd.concat([featureDF,oneHotADJ], axis =1, join = 'inner')
#Rename type column so when we turn the word list into a column it doesn't replace it.
df = df.rename({'type': 'myerTypes'}, axis=1)
df.head()

,myerTypes,EmojiFeature,ADJs,AvgSentLength,MostCommonWord,AvgChar,Avg TFIDF,vaderScore,CombineDict,Clean,...,able,strong,human,enfp,interested,entp,emotional,easy,young,certain
0,INFJ,"{'< happy >': False, '< laugh >': False, '< fr...",59,13.6,92,5.75,0.042,2,"({'< happy >': False, '< laugh >': False, '< f...",' and intj moments sportscenter not top ten...,...,0,0,0,1,0,1,0,0,0,0
1,ENTP,"{'< happy >': True, '< laugh >': False, '< fro...",104,11.8,112,5.52,0.023,2,"({'< happy >': True, '< laugh >': False, '< fr...",'I'm finding the lack of me in these posts ver...,...,1,0,0,0,0,1,0,1,0,0
2,INTP,"{'< happy >': True, '< laugh >': False, '< fro...",67,14.3,92,5.84,0.032,2,"({'< happy >': True, '< laugh >': False, '< fr...","'Good one _____ course, to which I say I k...",...,1,0,1,0,1,0,0,0,0,0
3,INTJ,"{'< happy >': False, '< laugh >': False, '< fr...",99,11.9,119,5.93,0.029,2,"({'< happy >': False, '< laugh >': False, '< f...","'Dear INTP, I enjoyed our conversation the o...",...,1,0,0,0,0,0,1,0,0,1
4,ENTJ,"{'< happy >': False, '< laugh >': True, '< fro...",67,11.5,97,5.88,0.032,2,"({'< happy >': False, '< laugh >': True, '< fr...",'You're fired.That's another silly misconcepti...,...,0,0,0,0,1,0,0,1,0,0


In [66]:
#Since Most Common Word(MCW) is same like ADJs we need to find out the particular word for that type

In [67]:
#Cleaner function that filters out stopwords, punctuations, and keeps english characters
stopwordlist = gensim.parsing.preprocessing.STOPWORDS
def cleaner(post):
    # remove words in < > or special symbols residuals
    special_symbols_remove = re.sub('\<[A-Za-z]+.[\s\w]*\>|\(|\)|\*|[A-Z]\/[A-Z]|\_+', '', post)
    wordlist = nltk.word_tokenize(special_symbols_remove)
    words_nonstop = [word.lower() for word in wordlist if word.lower() not in stopwordlist or word not in string.punctuation]
    words = [word for word in words_nonstop if len(word) > 3] # words that length than 3
    return words
#We need CLean this so we have a list of words for each post to compare later on
df['Cleaner'] = df['Clean'].apply(lambda post: cleaner(post))

In [68]:
#Create a word list of all the words from the corpus
all_word = []
for i in range(len(df['Clean'])):
    word = cleaner(df['Clean'][i])
    for j in word:
        all_word.append(j)

fd = nltk.FreqDist(all_word)      
most_common = fd.most_common(300)
most = [z[0] for z in most_common]
most

['that',
 'have',
 'with',
 'this',
 'like',
 'just',
 'what',
 'about',
 'think',
 'people',
 'your',
 'when',
 'they',
 'would',
 'know',
 'really',
 'more',
 'because',
 'from',
 'there',
 'some',
 'time',
 'very',
 'them',
 'feel',
 'much',
 'being',
 'been',
 'love',
 'other',
 'things',
 'good',
 'want',
 'something',
 'will',
 'most',
 'only',
 'also',
 'then',
 'could',
 'than',
 'well',
 'does',
 'always',
 'even',
 'type',
 'here',
 'someone',
 'make',
 'their',
 'myself',
 'never',
 'life',
 'find',
 'into',
 'thing',
 'though',
 'which',
 'going',
 'actually',
 'person',
 'sure',
 'infp',
 'right',
 'pretty',
 'were',
 'first',
 'friends',
 'need',
 'tongue',
 'playful',
 'sticking',
 'infj',
 'blowing',
 'cheeky',
 'same',
 'raspberry',
 'still',
 'intj',
 'work',
 'thought',
 'said',
 'where',
 'many',
 'should',
 'over',
 'friend',
 'take',
 'read',
 'around',
 'probably',
 'intp',
 'best',
 'thread',
 'sometimes',
 'years',
 'mean',
 'back',
 'those',
 'anything',
 'may

In [69]:
#Checking the words pos
nltk.pos_tag(most)

[('that', 'DT'),
 ('have', 'VBP'),
 ('with', 'IN'),
 ('this', 'DT'),
 ('like', 'JJ'),
 ('just', 'RB'),
 ('what', 'WP'),
 ('about', 'IN'),
 ('think', 'VBP'),
 ('people', 'NNS'),
 ('your', 'PRP$'),
 ('when', 'WRB'),
 ('they', 'PRP'),
 ('would', 'MD'),
 ('know', 'VB'),
 ('really', 'RB'),
 ('more', 'RBR'),
 ('because', 'IN'),
 ('from', 'IN'),
 ('there', 'EX'),
 ('some', 'DT'),
 ('time', 'NN'),
 ('very', 'RB'),
 ('them', 'PRP'),
 ('feel', 'VB'),
 ('much', 'JJ'),
 ('being', 'VBG'),
 ('been', 'VBN'),
 ('love', 'IN'),
 ('other', 'JJ'),
 ('things', 'NNS'),
 ('good', 'JJ'),
 ('want', 'VBP'),
 ('something', 'NN'),
 ('will', 'MD'),
 ('most', 'RBS'),
 ('only', 'RB'),
 ('also', 'RB'),
 ('then', 'RB'),
 ('could', 'MD'),
 ('than', 'IN'),
 ('well', 'RB'),
 ('does', 'VBZ'),
 ('always', 'RB'),
 ('even', 'RB'),
 ('type', 'NN'),
 ('here', 'RB'),
 ('someone', 'NN'),
 ('make', 'VBP'),
 ('their', 'PRP$'),
 ('myself', 'PRP'),
 ('never', 'RB'),
 ('life', 'NN'),
 ('find', 'VBP'),
 ('into', 'IN'),
 ('thing', 'NN'

In [70]:
#Checking the Words and ADJs
notADJ = 0
ADJ = 0
for k,v in nltk.pos_tag(most):
    if "JJ" not in v:
        notADJ +=1
    else:
        ADJ +=1
print('Words not Adj:', notADJ)
print('Adjs:', ADJ)

Words not Adj: 233
Adjs: 67


In [71]:
#We still have to remove the words from already in our dataframe column.
wordsInDF = df.columns.values.tolist()
#Column 10 is when the list of ADJ starts
adjsInDF = wordsInDF[10:]

In [72]:
#Comparing the most list to our adjsInDF and removing the adjs from most common
mostCommonList = [i for i in most if i not in adjsInDF]
#Check by using len
print("Length of mostCommonList:",len(mostCommonList))
#Note we still have ADJs in the list and it's not the full clean
tagged = nltk.pos_tag(most)
noAdjList = [x for x,y in tagged if 'JJ' not in y and x.isalpha()] #Cleaning out the JJ
print("New Length of word list with no Adj:", len(noAdjList))

Length of mostCommonList: 260
New Length of word list with no Adj: 233


In [73]:
#Applying the word list to each post using a function
def mostCommon(post):
    featureMost = {}
    postList = list(set(post))
    for i in noAdjList:
        if i in postList:
            featureMost[i] = 1
        else:
            featureMost[i] = 0
    return(featureMost)

df['MostCommon'] = df['Cleaner'].apply(lambda x: mostCommon(x))

In [74]:
#Covert the keys in MostCommon it it's own Columns
list_a = list(enumerate(df['MostCommon'][0].keys()))
for index,lab in list_a:
    df[lab] = df['MostCommon'].apply(lambda x: [v for k,v in x.items()][index])

In [75]:
#Convert EmojiFeature in its own Column with one hot encoding
list_b = list(enumerate(df['EmojiFeature'][0].keys()))
for index,lab in list_b:
    df[lab.strip('< >')] = df['EmojiFeature'].apply(lambda x: [v for k,v in x.items()][index])
    df[lab.strip('< >')] = df[lab.strip('< >')].apply(lambda x: 1 if x==True else 0)

In [76]:
#Breaking down myerTypes to each individuals types
charList =list(set(''.join(list(set(df["myerTypes"])))))
charList

['P', 'S', 'N', 'T', 'I', 'J', 'F', 'E']

In [77]:
#Function that checks if the letters are in the type and assign it to 1 or 0
def letter(myer, letter):
    if letter in myer:
        i = 1
    else:
        i = 0
    return i
for char in charList:
    df[char]= df['myerTypes'].apply(lambda x: letter(x,char))

In [78]:
##Droping E,S,T,J to reduce redundancy
df = df.drop(['E', 'S', 'T', 'J'], axis = 1)

In [79]:
#Renaming I, N, F, P
df.rename({'I': 'I(1)/E', 'P': 'P(1)/J','N': 'N(1)/S', 'F': 'F(1)/T'}, axis=1, inplace=True)

In [80]:
#Removing the unused columns for the Classification
df = df.drop(['EmojiFeature', 'ADJs', 'MostCommonWord', 'CombineDict', 'Clean', 'Cleaner', 'MostCommon'], axis = 1)

In [81]:
#Checking the names of the columns
df.columns.values

array(['myerTypes', 'AvgSentLength', 'AvgChar', 'Avg TFIDF', 'vaderScore',
       'good', 'happy', 'much', 'cheeky', 'playful', 'big', 'sure',
       'many', 'little', 'bad', 'different', 'best', 'great', 'new',
       'sad', 'hard', 'last', 'right', 'first', 'old', 'high', 'infp',
       'mean', 'infj', 'intj', 'welcome', 'intp', 'live', 'social',
       'true', 'least', 'thread', 'nice', 'give', 'wrong', 'real',
       'write', 'long', 'agree', 'able', 'strong', 'human', 'enfp',
       'interested', 'entp', 'emotional', 'easy', 'young', 'certain',
       'that', 'have', 'with', 'this', 'just', 'what', 'about', 'think',
       'people', 'your', 'when', 'they', 'would', 'know', 'really',
       'more', 'because', 'from', 'there', 'some', 'time', 'very', 'them',
       'feel', 'being', 'been', 'love', 'things', 'want', 'something',
       'will', 'most', 'only', 'also', 'then', 'could', 'than', 'well',
       'does', 'always', 'even', 'type', 'here', 'someone', 'make',
       'their', '

In [82]:
df.to_pickle("ModelOneHots")

In [59]:
len(df.columns.values)

357

In [60]:
df

,myerTypes,AvgSentLength,AvgChar,Avg TFIDF,vaderScore,good,happy,much,cheeky,playful,...,joyous,th_love,lovekitty,skellie,confuse,ghost,P(1)/J,N(1)/S,I(1)/E,F(1)/T
0,INFJ,13.6,5.75,0.042,2,1,0,1,1,1,...,0,0,0,0,0,0,0,1,1,1
1,ENTP,11.8,5.52,0.023,2,1,1,1,1,0,...,0,0,0,0,0,0,1,1,0,0
2,INTP,14.3,5.84,0.032,2,1,1,1,0,0,...,0,0,0,0,0,0,1,1,1,0
3,INTJ,11.9,5.93,0.029,2,0,0,1,0,1,...,0,0,0,0,0,0,0,1,1,0
4,ENTJ,11.5,5.88,0.032,2,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8670,ISFP,14.6,5.81,0.032,2,1,1,1,0,0,...,0,0,0,0,0,0,1,0,1,1
8671,ENFP,14.0,5.80,0.019,2,1,1,1,1,1,...,0,0,0,0,0,0,1,1,0,1
8672,INTP,9.6,5.99,0.033,2,1,0,0,1,1,...,0,0,0,0,0,0,1,1,1,0
8673,INFP,20.5,5.65,0.022,2,1,0,1,0,0,...,0,0,0,0,0,0,1,1,1,1
